# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [3]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='C:/Users/User/Desktop/chromedriver/chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break

    print('網頁更新中...')
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 疫情陰霾未除　陽明董座：四月起可望有趕單效應「有信心市場好轉」

▲陽明海運董事長謝志堅認為，預估四、五月起將有趕單效應，對今年市場好轉有信心。（圖／記者黃克翔攝）
記者林淑慧／台北報導
新冠肺炎疫情延燒，衝擊全球經貿網絡，陽明海運董事長謝志堅認為，若疫情於三月起好轉，海運訂單都還在，只是延後出貨，加上台商陸續復工，預估四、五月起將有趕單效應，對今年市場好轉有信心。
謝志堅指出，依據Alphaliner最新預測，2020年船噸供給成長率為3.5％ ，需求為2.2％，實際供給增加率恐因疫情延燒，另外，因為拆解與裝脫硫器船多，加上部分船隻延期交船而壓低，供給端預估可能再往下，但貨物需求仍在。
另外，自2016年韓進倒閉之後，各國船東呈現高度自制力，多家大型貨櫃船公司停止訂造新船，目前全球新船訂單已降至10.4％，近一年來也透過減併班、降 低航速來調節運力，穩定市場運價，另為了反應使用低硫油所增加的成本，各航線都有開始收取低硫油附加費，均能穩定市場。
----------------------------------------------------------------------

[1] 機器人理財卡在「半自動化」　上路2年多資產規模只有7.1億

▲根據金管會統計，截至去年底，機器人理財資產規模僅7.1億元。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
金管會在2017年開放「自動化投資顧問業務」（機器人理財），不過截至2019年12月底，機器人理財資產規模僅7.1億元。金管會對此表示，機器人理財是一項新技術，目前處於發展初期階段，仍看好未來發展，國內投資人也需花時間熟悉。
2017年6月，金管會開放「自動化投資顧問業務」（機器人理財），但這項新業務似乎不受投資人青睞，截至去年底，機器人理財資產規模僅7.1億元，與2017年底相較，1年只增加4家金融機構開辦，管理資產規模也僅增加1.78億元；金管會證期局官員強調，目前尚屬開辦初期階段，從外國經驗來看，未來還有很大發展空間。
近年來，隨著網際網路的普及與科技的發展，金融科技（FinTech）在全球金融業已引領一股創新風潮，例如電子支付、區塊鏈技術等，也包括透過演